In [119]:
import wti_env

In [120]:
import pandas as pd
import numpy as np

In [121]:
def create_merged_df(contract_type):
  spot = pd.read_csv("/Users/ethanhersch/Documents/Documents/Stanford/CS238/FinalProject/CS238_Project/data/oil_data/Cushing_OK_WTI_Spot_Price_FOB_sorted.csv")
  futures = pd.read_csv(f"/Users/ethanhersch/Documents/Documents/Stanford/CS238/FinalProject/CS238_Project/data/oil_data/NYMEX_futures/Cushing_OK_Crude_Oil_Future_Contract_{contract_type}.csv")

  df = pd.merge(spot, futures, on="Date", how="inner")
  df = df.rename(columns={"Price_y": "CL1"})
  df = df.rename(columns={"Price_x": "spot_price"})
  df[f"ret_CL1"] = np.log(df["CL1"].shift(-1) / df[f"CL1"])
  return df

In [155]:
create_merged_df(1)

/Users/ethanhersch/anaconda3/lib/python3.11/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


,Date,spot_price,CL1,ret_CL1
0,1986-01-02,25.56,25.56,0.015913
1,1986-01-03,26.00,25.97,0.022841
2,1986-01-06,26.53,26.57,-0.014023
3,1986-01-07,25.85,26.20,-0.010359
4,1986-01-08,25.87,25.93,-0.001158
...,...,...,...,...
9581,2024-04-01,84.54,83.71,0.017056
9582,2024-04-02,85.95,85.15,0.003283
9583,2024-04-03,86.22,85.43,0.013487
9584,2024-04-04,87.37,86.59,0.003689


In [147]:
def add_pipeline_weather_data(df, location):
  loc_df = pd.read_csv(f"/Users/ethanhersch/Documents/Documents/Stanford/CS238/FinalProject/CS238_Project/data/processed_data/incoming/{location}_incoming.csv")

  loc_df = loc_df.rename(columns={"mean_temp_K": f"{location}_temp"})
  loc_df = loc_df.rename(columns={"total_precip_mm": f"{location}_precip"})
  df = pd.merge(df, loc_df, on="Date", how="left") # use left merge to use dates from left df
  return df

In [176]:
locations = ["alaska", "bakken", "western_canada", "midwest_corridor", "ok_hub", "permian", "rockies", "texas_gulf"]

df = create_merged_df(4)
for loc in locations:
  print(loc)
  df = add_pipeline_weather_data(df, loc)

df

alaska
bakken
western_canada
midwest_corridor
ok_hub
permian
rockies
texas_gulf


,Date,spot_price,CL1,ret_CL1,alaska_temp,alaska_precip,bakken_temp,bakken_precip,western_canada_temp,western_canada_precip,midwest_corridor_temp,midwest_corridor_precip,ok_hub_temp,ok_hub_precip,permian_temp,permian_precip,rockies_temp,rockies_precip,texas_gulf_temp,texas_gulf_precip
0,1986-01-02,25.56,23.21,0.017511,248.181843,2.670807,267.462871,4.618883,259.668224,2.426405,272.636515,1.420228,280.687172,0.000855,282.793781,0.000860,269.915554,1.112069,284.128822,0.489698
1,1986-01-03,26.00,23.62,0.023432,248.961898,3.482124,262.755624,0.840877,257.547465,0.577723,272.069060,0.977935,279.298853,0.000861,282.357327,0.001564,268.373695,1.647863,285.753964,0.373387
2,1986-01-06,26.53,24.18,-0.017943,258.344380,2.313171,254.950851,0.518769,255.220885,0.686386,265.290904,0.105560,278.176843,0.159463,281.672159,0.020884,267.812953,2.884952,280.767185,0.613318
3,1986-01-07,25.85,23.75,-0.014847,256.710557,0.916802,257.554692,0.141348,261.864550,0.173848,260.424234,0.100934,274.819601,0.444359,277.907672,5.020631,266.607039,1.334517,283.705630,3.066794
4,1986-01-08,25.87,23.40,-0.005571,255.486960,1.147838,268.006027,0.000200,268.871573,0.870102,263.865249,0.010327,271.596204,0.038997,271.975373,2.905248,266.647574,0.000956,278.790667,6.579200
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9583,2024-04-01,84.54,81.12,0.014927,266.456916,2.907795,273.980103,0.103339,274.328369,0.498237,282.581676,6.058377,294.530965,3.144950,294.180991,1.217820,274.246254,3.640875,296.917074,0.293375
9584,2024-04-02,85.95,82.34,0.006537,262.338132,1.333529,277.604200,0.008144,276.605366,0.305583,280.158687,11.657735,287.310766,7.704964,287.198399,0.300033,275.216672,0.248428,295.829715,2.786975
9585,2024-04-03,86.22,82.88,0.013780,259.954487,0.559277,278.807268,0.018436,278.190696,2.470537,278.711498,2.718310,285.542092,0.003279,287.183338,0.018029,278.428109,0.030500,291.111963,0.512906
9586,2024-04-04,87.37,84.03,0.002496,263.308430,1.070032,281.382040,0.020203,274.833164,2.265301,279.489586,0.431029,287.226755,0.001722,291.200820,0.001722,280.880427,0.080914,291.564768,0.001741


In [177]:
def add_producing_weather_data(df, location):
  loc_df = pd.read_csv(f"/Users/ethanhersch/Documents/Documents/Stanford/CS238/FinalProject/CS238_Project/data/processed_data/producing_locations/{location}_producing.csv")

  loc_df = loc_df.rename(columns={"mean_temp_K": f"{location}_temp_producing"})
  loc_df = loc_df.rename(columns={"total_precip_mm": f"{location}_precip_producing"})
  df = pd.merge(df, loc_df, on="Date", how="left")
  return df

In [178]:
locations = ["bakken", "canada", "eagle", "central_kansas", "n_alaska", "nc_ok", "ne_ok", "nw_ok", "permian", "s_alaska", "saskatchewan", "se_kansas", "sw_kansas"]

for loc in locations:
  df = add_producing_weather_data(df, loc)

In [179]:
df.to_csv("/Users/ethanhersch/Documents/Documents/Stanford/CS238/FinalProject/CS238_Project/src/wti_4_data.csv", index=False)